<a href="https://colab.research.google.com/github/conghcmus/Fake_News_Detection/blob/master/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Khai báo thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, LSTM, Input, Conv1D, MaxPooling1D, Activation, BatchNormalization, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate, Add
from tensorflow.keras.models import Model

In [ ]:
# bert_preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

## Download pretrained model

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip glove.840B.300d.zip

In [ ]:
# !wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip'
# !unzip 'wiki-news-300d-1M.vec.zip'

In [ ]:
# !cp 'wiki-news-300d-1M.vec' '/content/drive/MyDrive/seminar/pretrained model/'

cp: cannot create regular file '/content/drive/MyDrive/seminar/pretrained model/': No such file or directory


In [ ]:
!ls

cc.en.300.bin.gz    drive	 wiki-news-300d-1M.vec
cc.en.300.bin.gz.1  sample_data  wiki-news-300d-1M.vec.zip


## ISOT Fake News Dataset
Link: https://onlineacademiccommunity.uvic.ca/isot/2022/11/27/fake-news-detection-datasets/

### Đọc dữ liệu

In [ ]:
# !wget https://onlineacademiccommunity.uvic.ca/isot/wp-content/uploads/sites/7295/2023/03/News-_dataset.zip
# !unzip News-_dataset.zip

In [ ]:
# !cp 'Fake.csv' '/content/drive/MyDrive/seminar/datasets/ISOT Fake News Dataset/'
# !cp 'True.csv' '/content/drive/MyDrive/seminar/datasets/ISOT Fake News Dataset/'
# !cp 'glove.840B.300d.txt' '/content/drive/MyDrive/seminar/pretrained model/'

In [ ]:
%cd '/content/drive/MyDrive/seminar/'

/content/drive/MyDrive/seminar


In [ ]:
True_data = pd.read_csv("datasets/ISOT Fake News Dataset/True.csv")
Fake_data = pd.read_csv("datasets/ISOT Fake News Dataset/Fake.csv")
True_data['label'] = 0
Fake_data['label'] = 1
print(True_data.shape)
print(Fake_data.shape)

(21417, 5)
(23481, 5)


In [ ]:
df_concat = pd.concat([True_data, Fake_data], axis=0)
df_concat.shape

(44898, 5)

### Text processing

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
sw = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Defining contractions
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [ ]:
# Remove contraction
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [ ]:
# Function remove_tags to handles HTML tags
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
def cleaned_text(df):
    # convert lower
    df['cleaned_text'] = df['text'].str.lower()
    # Remove html tags
    df['cleaned_text'] = df['cleaned_text'].apply(remove_html_tags)
    # remove punctuations
    df['cleaned_text'] = df['cleaned_text'].str.replace('[^\w\s]', '', regex=True)
    # remove numeric characters
    df['cleaned_text'] = df['cleaned_text'].str.replace('\d', '', regex=True)
    # remove contractions
    df['cleaned_text'] = df['cleaned_text'].apply(remove_contractions)
    # remove stopwords
    df['cleaned_text'] = df['cleaned_text'].apply(lambda row: " ".join(x for x in row.split() if x not in sw))
    # remove new line
    df['cleaned_text'] = df['cleaned_text'].apply(lambda row: row.replace('\n', ''))

    return df


In [ ]:
df_concat = cleaned_text(df_concat)
df_concat[['text', 'cleaned_text']].iloc[0]

text            WASHINGTON (Reuters) - The head of a conservat...
cleaned_text    washington reuters head conservative republica...
Name: 0, dtype: object

In [ ]:
df_concat = df_concat.sample(frac = 1)
df_concat.head(10)

,title,text,subject,date,label,cleaned_text
10462,PRESIDENT TRUMP AND MELANIA Arrive in Poland t...,The President and First Lady touched down in W...,politics,"Jul 5, 2017",1,president first lady touched warsaw poland tod...
14561,Botswana's Khama tells Mugabe to go,JOHANNESBURG (Reuters) - Zimbabwe President Ro...,worldnews,"November 17, 2017",0,johannesburg reuters zimbabwe president robert...
19166,Germany's jubilant far-right has Merkel in its...,BERLIN (Reuters) - Swept into parliament by th...,worldnews,"September 24, 2017",0,berlin reuters swept parliament germans angere...
10375,"Aiming to sidestep Apple dispute, Obama makes ...","AUSTIN, Texas (Reuters) - U.S. President Barac...",politicsNews,"March 11, 2016",0,austin texas reuters us president barack obama...
4566,Montel Williams DESTROYS Pro-Trump Pastor In ...,Montel Williams isn t afraid to express his ab...,News,"September 19, 2016",1,montel williams afraid express absolute disgus...
3385,"Trump BAILS On Teleprompter, Insults His Own ...",Despite not having any time to attend critical...,News,"December 16, 2016",1,despite time attend critical press conferences...
11519,"Sickening Reason COP-HATING, RACIST 49’ers QB ...",It looks like veteran San Francisco 49ers quar...,politics,"Mar 2, 2017",1,looks like veteran san francisco ers quarterba...
16830,"Suicide bombers attack two Afghan mosques, at ...",KABUL (Reuters) - Suicide bombers attacked two...,worldnews,"October 20, 2017",0,kabul reuters suicide bombers attacked two mos...
18245,WATCH: SARAH HUCKABEE-SANDERS Mocks Media For ...,Sarah Huckabee Sanders has taken control of th...,left-news,"Jul 28, 2017",1,sarah huckabee sanders taken control media lon...
19083,WOW! TUCKER AND JESSE Destroy The Liberal Kook...,https://www.youtube.com/watch?v=J4LjxrOfEF8,left-news,"Feb 26, 2017",1,httpswwwyoutubecomwatchvjljxrofef


### Build vocabulary

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df_concat['cleaned_text'])

In [ ]:
len(tokenizer.word_index)

211546

In [ ]:
# vocab_size = len(tokenizer.word_index) + 1
sequence_length = 1000
embedding_dim = 300

### Chia tập train và test

In [ ]:
train_sentences, test_sentences, train_label, test_label = train_test_split(
    df_concat['cleaned_text'], df_concat['label'], test_size=0.2, stratify=df_concat['label']
    )

In [ ]:
train_labels = np.array(train_label)
test_label = np.array(test_label)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
padded_train_sequences = pad_sequences(train_sequences, maxlen=sequence_length, truncating='post', padding='post')
padded_test_sequences = pad_sequences(test_sequences, maxlen=sequence_length, truncating='post', padding='post')

In [ ]:
padded_train_sequences

array([[ 2323,   372,    58, ...,     0,     0,     0],
       [   36,    12,     3, ...,     0,     0,     0],
       [  207,    27, 14886, ...,     0,     0,     0],
       ...,
       [  586, 10732,  7077, ...,     0,     0,     0],
       [ 7527,  1940,  1201, ...,     0,     0,     0],
       [   28,   202,  2469, ...,     0,     0,     0]], dtype=int32)

In [ ]:
padded_train_sequences.shape

(35918, 1000)

### Build model

#### TF-IDF + Linear SVM

In [ ]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers using tfidf
    ("SVC", SVC()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_label)

In [ ]:
# Evaluate baseline model
base_line_score = model_0.score(test_sentences, test_label)
print(f"accuracy of baseline model: {base_line_score*100:.2f}")

#### TF-IDF + Decision tree

In [ ]:
# Create tokenization and modelling pipeline
model_1 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("decision tree", DecisionTreeClassifier())
])

# Fit the pipeline to the training data
model_1.fit(train_sentences, train_label)

In [ ]:
# Evaluate baseline model
base_line_score = model_1.score(test_sentences, test_label)
print(f"accuracy of baseline model: {base_line_score*100:.2f}")

#### fastText + CNN

In [ ]:
from gensim.models import FastText

# # Original fasttext embeddings from https://fasttext.cc/
# ft = FastText.load_fasttext_format("fasttext_embedding.bin")
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

embedding_matrix = load_vectors("wiki-news-300d-1M.vec")
embedding_matrix[0:2]

In [ ]:
vocab_size = 10000
input_layer = Input(shape=(sequence_length,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
conv1d_1 = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding_layer)
conv1d_2 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding_layer)
conv1d_3 = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
maxpool_1 = MaxPooling1D(pool_size=5)(conv1d_1)
maxpool_2 = MaxPooling1D(pool_size=5)(conv1d_2)
maxpool_3 = MaxPooling1D(pool_size=5)(conv1d_3)
concatenated = Concatenate(axis=1)([maxpool_1, maxpool_2, maxpool_3])
conv1d_4 = Conv1D(filters=128, kernel_size=5, activation='relu')(concatenated)
maxpool_4 = MaxPooling1D(pool_size=5)(conv1d_4)
conv1d_5 = Conv1D(filters=128, kernel_size=5, activation='relu')(maxpool_4)
maxpool_5 = MaxPooling1D(pool_size=5)(conv1d_5)
global_maxpool = GlobalMaxPooling1D()(maxpool_5)
dense_1 = Dense(128, activation='relu')(global_maxpool)
output_layer = Dense(1, activation='sigmoid')(dense_1)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1000)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1000, 300)            3000000   ['input_2[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 998, 128)             115328    ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 997, 128)             153728    ['embedding[0][0]']           
                                                                                              

#### GloVe + ResNet

In [ ]:
%%time
embeddings_dictionary = dict()
glove_file = open('pretrained model/glove.840B.300d.txt', encoding="utf8")
for line in glove_file:
    records = line.split(' ')
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[str(word)] = vector_dimensions
glove_file.close()

CPU times: user 2min 36s, sys: 6.5 s, total: 2min 42s
Wall time: 3min 1s


In [ ]:
embeddings_dictionary.__len__()

2196016

In [ ]:
# Tạo Embedding Matrix có 300 columns
# Bao gồm 300-dimensional GloVe word embeddings cho tất cả các từ trong vocab.
embedding_matrix = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
vocab_size = 10000
input_layer = Input(shape=(sequence_length,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
conv1d_1 = Conv1D(filters=256, kernel_size=3, activation='relu')(embedding_layer)
conv1d_2 = Conv1D(filters=256, kernel_size=4, activation='relu')(embedding_layer)
conv1d_3 = Conv1D(filters=256, kernel_size=5, activation='relu')(embedding_layer)
maxpool_1 = MaxPooling1D(pool_size=5)(conv1d_1)
maxpool_2 = MaxPooling1D(pool_size=5)(conv1d_2)
maxpool_3 = MaxPooling1D(pool_size=5)(conv1d_3)
concatenated = Concatenate(axis=1)([maxpool_1, maxpool_2, maxpool_3])
maxpool_4 = MaxPooling1D(pool_size=5)(concatenated)
conv1d_5 = Conv1D(filters=256, kernel_size=1, activation='relu')(maxpool_4)
batchnorm_1 = BatchNormalization()(conv1d_5)
activation_1 = Activation(activation='relu')(batchnorm_1)
conv1d_6 = Conv1D(filters=256, kernel_size=1, activation='relu')(activation_1)
batchnorm_2 = BatchNormalization()(conv1d_6)
activation_2 = Activation(activation='relu')(batchnorm_2)
conv1d_7 = Conv1D(filters=256, kernel_size=1, activation='relu')(activation_2)
batchnorm_3 = BatchNormalization()(conv1d_7)

add = Add()([maxpool_4, batchnorm_3])
global_avgpool = GlobalAveragePooling1D()(add)
output_layer = Dense(1, activation='sigmoid')(global_avgpool)

resnet_model = Model(inputs=input_layer, outputs=output_layer)

resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 1000)]               0         []                            
                                                                                                  
 embedding_13 (Embedding)    (None, 1000, 300)            3000000   ['input_15[0][0]']            
                                                                                                  
 conv1d_54 (Conv1D)          (None, 998, 256)             230656    ['embedding_13[0][0]']        
                                                                                                  
 conv1d_55 (Conv1D)          (None, 997, 256)             307456    ['embedding_13[0][0]']        
                                                                                           

#### GloVe + Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=sequence_length))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         63464100  
                                                                 
 bidirectional (Bidirection  (None, 1000, 512)         1140736   
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 512)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 64670629 (246.70 MB)
Trainable params: 646

In [ ]:
%%time

model.fit(padded_train_sequences, train_labels, batch_size=64, epochs=6)

Epoch 1/6
562/562 [==============================] - 220s 368ms/step - loss: 0.0225 - accuracy: 0.9923
Epoch 2/6
118/562 [=====>........................] - ETA: 2:30 - loss: 3.2662e-04 - accuracy: 1.0000

## Fake news dataset
**Link:** https://www.kaggle.com/competitions/fake-news/overview <br>
**Dataset Description** <br>

**train.csv:** A full training dataset with the following attributes:

- **id:** unique id for a news article
- **title:** the title of a news article
- **author:** author of the news article
- **text:** the text of the article; could be incomplete
- **label:** a label that marks the article as potentially unreliable
    - 1: unreliable
    - 0: reliable
    
**test.csv:** A testing training dataset with all the same attributes at train.csv without the label.

In [ ]:
Fake_news_data = pd.read_csv("datasets/Fake news dataset/train.csv")
print(Fake_news_data.shape)
Fake_news_data.head(10)

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [ ]:
data_cleaned = cleaned_text(Fake_news_data)
data_cleaned[['text', 'cleaned_text']].iloc[0]

TypeError: ignored

## Fake or real news dataset
This is the repo hosting the data for my fake vs real news project. The data is in a zipped csv file and contains almost 11000 articles tagged as either real or fake. <br>
**Link:** https://github.com/joolsa/fake_real_news_dataset/tree/master

In [ ]:
Fake_or_real_news_data = pd.read_csv("datasets/Fake or real news dataset/fake_or_real_news_data.csv")
print(Fake_or_real_news_data.shape)
Fake_or_real_news_data.head(10)

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


## Fake news detection dataset
**Link:** https://www.kaggle.com/datasets/jruvika/fake-news-detection

In [ ]:
Fake_news_detection_data = pd.read_csv("datasets/Fake news detection dataset/fake_news_detection_data.csv")
print(Fake_news_detection_data.shape)
Fake_news_detection_data.head(10)

NameError: ignored